In [ ]:
import requests
import pandas as pd
import json

def get_stock_data(symbols, api_key):
    stock_data = []
    for symbol in symbols:
        url = f"https://yfapi.net/v6/finance/quote?symbols={symbol}"
        headers = {"x-api-key": api_key}
        response = requests.get(url, headers=headers).json()
        quote = response.get("quoteResponse", {}).get("result", [{}])[0]
        stock_data.append({
            'Stock Ticker': symbol,
            'Company': quote.get('longName', 'Unavailable'),
            'Current Market Price': quote.get('regularMarketPrice', 'Unavailable')
        })
    return stock_data

def get_additional_data(symbol, module, api_key):
    url = f"https://yfapi.net/v11/finance/quoteSummary/{symbol}?modules={module}"
    headers = {"x-api-key": api_key}
    response = requests.get(url, headers=headers).json()
    summary = response.get("quoteSummary", {}).get("result", [{}])[0]
    data = {
        '52 Week High': summary.get("summaryDetail", {}).get("fiftyTwoWeekHigh", {}).get("raw", 'Unavailable'),
        '52 Week Low': summary.get("summaryDetail", {}).get("fiftyTwoWeekLow", {}).get("raw", 'Unavailable'),
        'Return on Assets': summary.get("financialData", {}).get("returnOnAssets", {}).get("raw", 'Unavailable')
    }
    return pd.DataFrame([data])

def get_trending_stocks(api_key):
    url = "https://yfapi.net/v1/finance/trending/US"
    headers = {"x-api-key": api_key}
    response = requests.get(url, headers=headers).json()
    trending_symbols = [item["symbol"] for item in response.get("finance", {}).get("result", [{}])[0].get("quotes", [])]
    trending_data = get_stock_data(trending_symbols, api_key)
    return pd.DataFrame(trending_data)

if __name__ == "__main__":
    api_key = input("Enter your Yahoo Finance API key: ")

    symbols = input("Enter stock symbols separated by commas: ").split(',')
    stock_data = get_stock_data(symbols, api_key)
    for stock in stock_data:
        print(f"Stock Ticker: {stock['Stock Ticker']}, Company: {stock['Company']}, Current Market Price: ${stock['Current Market Price']}")

    symbol = input("Enter a stock symbol to fetch additional data: ")
    module = input("Choose a module (summaryDetail, financialData): ")
    df = get_additional_data(symbol, module, api_key)
    print("\nAdditional Data:")
    print(df)

    trending_stocks = get_trending_stocks(api_key)
    print("\nTrending Stocks:")
    print(trending_stocks)
